In [1]:
import pandas as pd
import wandb
import numpy as np

# First run

In [2]:
api_wandb = wandb.Api()

run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/mnipfdm0').scan_history())

In [3]:
run

,_step,progressivity,vkdnw_lambda_1,vkdnw_lambda_5,vkdnw_lambda_8,vkdnw_dim,complexity,vkdnw_lambda_7,arch,trainability,...,vkdnw_lambda_0,vkdnw_chisquare,flops,num_layers,vkdnw_lambda_4,_timestamp,vkdnw_lambda_3,_runtime,model_size,vkdnw_lambda_2
0,0,-Infinity,0.000244,0.194629,7.421612,244,381031544.0,1.810366,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.190722,...,8.016209e-07,190.459459,381031544.0,14,0.068927,1.729607e+09,0.019113,17.287027,3627705.0,0.003603
1,1,-0.017285,0.000006,0.093808,4.455290,248,434595992.0,1.264581,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.186515,...,1.113927e-07,143.468354,434595992.0,14,0.021177,1.729607e+09,0.003287,18.087931,3688617.0,0.000377
2,2,-0.709576,0.000057,0.790009,55.465851,270,402400248.0,10.776753,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.185046,...,9.500461e-06,116.229508,402400248.0,15,0.140649,1.729607e+09,0.024333,18.874209,3652009.0,0.001123
3,3,-0.338296,0.000059,0.523543,13.819300,274,410379800.0,4.165202,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-0.175802,...,2.507468e-06,157.880597,410379800.0,15,0.115631,1.729607e+09,0.014269,19.651100,3679761.0,0.000829
4,4,-0.247345,0.000061,0.276663,11.580867,280,417704712.0,2.290114,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.239618,...,1.729702e-07,176.583333,417704712.0,14,0.097906,1.729607e+09,0.023244,20.503012,3675097.0,0.004763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76246,76246,-0.789663,0.000065,2.150551,226.252106,320,226964764.0,63.217213,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",-0.377860,...,7.038095e-07,120.836364,226964764.0,14,0.445777,1.729693e+09,0.117521,86387.795209,1126625.0,0.009473
76247,76247,-Infinity,0.000005,1.528132,59.838116,274,388668292.0,16.284239,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.268079,...,1.852322e-07,113.655172,388668292.0,13,0.023683,1.729693e+09,0.000678,86388.898207,3500993.0,0.000048
76248,76248,-0.329206,0.000002,2.289304,69.226440,320,396934984.0,13.941657,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.341497,...,1.763321e-07,120.461538,396934984.0,14,0.731657,1.729693e+09,0.137673,86390.667794,3063665.0,0.006313
76249,76249,-Infinity,0.000006,1.378431,40.040604,274,438466012.0,10.738747,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.236537,...,9.194015e-07,147.724138,438466012.0,13,0.333114,1.729693e+09,0.069614,86391.824915,3775769.0,0.002764


In [8]:
for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')

In [9]:
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

In [10]:
run.shape

(76251, 28)

In [18]:
run['vkdnw_rank'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True)

In [36]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('az_nas', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
19012,19012.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K5(32,...",449037468.0,322.0,15.0,3685273.0,44.805483,41972.0,21.869541,44.239056,44.209899,44.321068


In [34]:
run.loc[run['vkdnw_dim']==346, ['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
31828,31828.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",449079804.0,346.0,15.0,2313505.0,43.579347,76177.0,22.465845,43.912776,43.73765,44.455743


In [42]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
64775,64775.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",448750524.0,326.0,15.0,3782881.0,44.752932,66546.0,22.283814,44.661257,44.687104,44.652615


In [44]:
#run.loc[run['arch']=='SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,40,2,24,1)SuperResIDWE1K5(40,64,2,48,4)SuperResIDWE1K5(64,192,2,144,5)SuperResIDWE6K5(192,176,2,176,3)SuperConvK1BNRELU(176,2048,1,1)', ]

# Other runs

In [4]:
import pandas as pd
import wandb
import numpy as np

api_wandb = wandb.Api()

In [9]:
# TOP_vkdnw_b_flops480M_layers18
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/2i17bohj').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
86189,86189.0,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K3(48,...",479999294.0,396.0,18.0,3264657.0,45.886779,22.822753,45.730079,45.746149,45.747762


In [8]:
# TOP_vkdnw_b_flops480M_layers15
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/73qqqhqt').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
89488,89488.0,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE2K5(80,...",479470486.0,346.0,15.0,2046209.0,45.117084,23.018287,45.124225,45.970446,45.1762


# New

In [18]:
# TOP_vkdnw_entropy_flops480M_layers14, random params
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/7ov6q9qv').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)

In [19]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('rank', ascending=False).head(10)
'SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K3(72,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K5(120,176,1,192,3)SuperResIDWE2K7(176,144,2,168,5)SuperConvK1BNRELU(144,2048,1,1)'

'SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K3(72,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K5(120,176,1,192,3)SuperResIDWE2K7(176,144,2,168,5)SuperConvK1BNRELU(144,2048,1,1)'

In [21]:
# TOP_vkdnw_entropy_flops480M_layers14_2
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('az_nas', ascending=False).head(20)
'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K7(120,152,1,240,3)SuperResIDWE6K5(152,112,2,112,5)SuperConvK1BNRELU(112,2048,1,1)'

,_step,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank
40842,40842.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K5(24,...",479792220.0,306.0,44.537932,-0.397331,127.652835,-0.163735,30140
61176,61176.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",479721660.0,310.0,44.500797,-0.396688,126.907147,-0.167757,37962
49181,49181.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.489621,-0.398352,127.517826,-0.169830,34025
47485,47485.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.487284,-0.396580,127.662912,-0.172456,43809
66416,66416.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",479721660.0,310.0,44.483574,-0.395120,126.772448,-0.170604,47624
41941,41941.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.480284,-0.391624,127.548190,-0.173990,56327
54654,54654.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",479690300.0,310.0,44.480045,-0.396493,127.939076,-0.172930,50954
19907,19907.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.474061,-0.395050,127.888495,-0.174318,30852
67007,67007.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.473239,-0.397807,127.341166,-0.171584,44537
61232,61232.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",479690300.0,310.0,44.472919,-0.397671,127.784373,-0.172040,54382


In [22]:
# TOP_vkdnw_entropy_flops480M_layers14_3
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('vkdnw_dim', ascending=False).head(20)
'SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE2K5(120,120,1,192,3)SuperResIDWE4K5(120,88,2,168,5)SuperConvK1BNRELU(88,2048,1,1)'

,_step,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank
17839,17839.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",467638162.0,346.0,41.876212,-0.531196,125.979684,-0.394221,69583
40776,40776.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",450946900.0,346.0,41.356301,-0.481966,131.475983,-0.435285,68067
26163,26163.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",441689918.0,346.0,42.316317,-0.401681,124.427164,-0.354214,64168
32983,32983.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",407537702.0,346.0,41.073718,-0.716819,121.051006,-0.357186,66750
32981,32981.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",466074082.0,346.0,42.709279,-0.400832,125.577932,-0.347882,69329
62544,62544.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",372944878.0,346.0,40.386910,-0.496537,121.454497,-0.384662,65745
47436,47436.0,"SuperConvK3BNRELU(3,8,2,1)SuperResIDWE1K3(8,40...",445578754.0,346.0,42.375321,-0.405643,127.861090,-0.376137,67260
5728,5728.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE4K3(32,...",472296494.0,346.0,37.297787,-inf,-inf,-0.323325,64102
5727,5727.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",459772780.0,346.0,39.555881,-0.980489,125.210986,-0.479805,68463
22952,22952.0,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K3(48,...",407410204.0,346.0,40.315074,-0.868098,118.106163,-0.392311,70070


In [4]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/uhoduwim').scan_history()) # VKDNW_EVOLUTION_FLOPS450000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,64,2,40,1)SuperResIDWE2K3(64,96,2,80,2)SuperResIDWE1K3(96,80,2,96,3)SuperResIDWE4K3(80,80,1,56,5)SuperResIDWE1K7(80,112,2,144,1)SuperResIDWE6K5(112,112,1,112,4)SuperConvK1BNRELU(112,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
71531,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.513380,-0.315760,144.089600,-0.268627,93879,56.963142
80779,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.530039,-0.313289,144.037058,-0.256952,92256,56.962361
85921,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.522489,-0.315573,144.455469,-0.271492,92300,56.955288
97574,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.516547,-0.316221,144.199271,-0.269263,91833,56.944274
96074,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.511030,-0.309874,143.831758,-0.265873,92303,56.943862
67960,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K7(64,...",448705248.0,392.0,45.567712,-0.326434,145.714550,-0.273469,86541,56.936086
97550,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449943380.0,396.0,45.514997,-0.299997,144.395626,-0.274173,91200,56.935807
75860,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449022278.0,396.0,45.476511,-0.310464,143.002159,-0.248483,94260,56.930323
78668,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",449022278.0,396.0,45.470821,-0.322040,143.256341,-0.258885,94466,56.926816
93405,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K5(64,...",448204664.0,396.0,45.483262,-0.325344,144.279970,-0.270749,93289,56.926719


In [5]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/il38fg43').scan_history()) # VKDNW_EVOLUTION_FLOPS450000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,40,2,32,1)SuperResIDWE1K5(40,80,2,72,2)SuperResIDWE4K7(80,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
45315,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.849709,-0.071986,128.060867,-0.327774,62288,56.889233
56879,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.826427,-0.070430,127.996855,-0.330729,62258,56.865470
94115,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.886136,-0.072719,128.085451,-0.322321,56732,56.832230
91664,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.797065,-0.080448,128.061676,-0.328387,61211,56.819148
86436,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.761602,-0.081376,127.980293,-0.331026,63391,56.818679
71885,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.791200,-0.070847,127.936033,-0.330552,60578,56.802887
60347,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.751488,-0.080362,127.762275,-0.324682,62968,56.801870
79229,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.822102,-0.079582,128.158788,-0.327177,58317,56.795751
87648,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.775434,-0.080739,127.973894,-0.330540,60849,56.791585
95486,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.752133,-0.081508,127.936150,-0.329528,62010,56.787184


In [6]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/ax1qckwo').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,40,2,40,1)SuperResIDWE1K3(40,96,2,96,2)SuperResIDWE4K7(96,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
88298,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.919450,-0.005563,129.401199,-0.314891,51649,56.771676
95786,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.914185,-0.007558,129.272707,-0.311299,51062,56.754981
60494,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.917586,-0.008506,129.479824,-0.313121,50792,56.753080
84647,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.943584,-0.006968,129.568012,-0.313660,48051,56.723602
72080,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.939005,-0.005929,129.495090,-0.314291,47794,56.713660
88486,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.903008,-0.006568,129.436402,-0.316091,49395,56.710613
92005,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.924352,-0.007248,129.525171,-0.314753,47685,56.696725
84710,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.891349,-0.006715,129.450716,-0.316896,49197,56.694937
75117,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.854507,-0.006092,129.453850,-0.319247,50999,56.694069
62632,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.839231,-0.008846,129.354128,-0.316961,51246,56.683624


In [16]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/592dqlhj').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,40,2,40,1)SuperResIDWE1K3(40,56,2,64,5)SuperResIDWE4K7(56,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
55557,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.873608,-0.111259,150.575394,-0.274803,58976,56.858494
83128,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.878883,-0.113637,150.628929,-0.273752,58251,56.851400
72361,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.865552,-0.110750,150.603093,-0.276053,58824,56.847857
18241,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.840663,-0.116069,150.590804,-0.276115,60300,56.847750
96089,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.851303,-0.116572,150.607646,-0.274938,59516,56.845303
40719,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.863732,-0.114722,150.522379,-0.272747,58595,56.842137
82043,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.835400,-0.111546,150.461509,-0.276235,60182,56.840528
96252,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.840100,-0.112792,150.693312,-0.277675,59794,56.838761
25684,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.849746,-0.111478,150.439522,-0.274094,59091,56.836580
46796,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.852149,-0.114584,150.493744,-0.274009,58829,56.834539


In [5]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/47mm5i5f').scan_history()) # VKDNW_EVOLUTION_FLOPS520000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run.loc[run['flops']<480000000, ['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,40,2,32,1)SuperResIDWE2K7(40,72,2,64,2)SuperResIDWE4K7(72,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)' # TOP

'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,56,2,48,2)SuperResIDWE1K3(56,96,2,128,1)SuperResIDWE2K5(96,112,2,120,3)SuperResIDWE4K5(112,112,2,112,3)SuperResIDWE4K5(112,112,1,112,4)SuperConvK1BNRELU(112,2048,1,1)' # TOP 4

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
21145,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479114648.0,274.0,42.120346,-0.092956,127.517635,-0.327002,26983,52.323308
36874,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",472030424.0,274.0,42.061262,-0.085270,128.225694,-0.332450,28057,52.303256
36691,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",466310360.0,274.0,42.035831,-0.075512,128.407858,-0.332190,28774,52.303058
14529,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,...",471643716.0,298.0,41.711740,-0.161149,123.448049,-0.317673,36767,52.224096
40971,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",468718808.0,274.0,42.030214,-0.096070,128.422935,-0.332836,26549,52.216962
11288,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",459416158.0,310.0,41.561901,-0.206044,125.708663,-0.321939,41891,52.204727
21220,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",469622662.0,310.0,41.523937,-0.351454,125.942455,-0.321544,42493,52.181031
10212,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,...",471643716.0,298.0,41.642727,-0.171950,123.668597,-0.325461,37456,52.173649
45931,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",457157552.0,274.0,42.001464,-0.094357,129.750493,-0.331660,26098,52.171078
19605,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K5(24,...",475963654.0,310.0,41.547360,-0.343593,125.096295,-0.323049,40357,52.152880


In [6]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/u6n06lkq').scan_history()) # VKDNW_EVOLUTION_FLOPS520000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run.loc[run['flops']<480000000, ['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,24,2,32,3)SuperResIDWE4K7(24,40,2,48,1)SuperResIDWE2K7(40,120,2,96,3)SuperResIDWE2K7(120,120,1,96,4)SuperResIDWE2K7(120,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)' # TOP

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
25592,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",475734040.0,388.0,40.658166,-0.262953,149.075095,-0.335206,29634,50.954843
27879,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K5(96,...",471493580.0,388.0,40.633798,-0.359137,144.746202,-0.290686,29485,50.925435
27470,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...",475948660.0,392.0,40.600576,-0.405251,142.360233,-0.279160,30362,50.921523
19630,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K7(96,...",479188638.0,392.0,40.587346,-0.574870,145.356719,-0.279303,30457,50.911417
33746,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",473143704.0,364.0,40.923801,-0.213260,154.081091,-0.315414,21498,50.899516
13911,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",471293072.0,388.0,40.654686,-0.207170,151.711402,-0.344474,27696,50.883729
24365,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",468827000.0,384.0,40.685900,-0.239196,152.438479,-0.336260,26600,50.874567
27121,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",475453368.0,380.0,40.739191,-0.274071,150.902516,-0.332096,24678,50.852858
24401,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",468827000.0,384.0,40.667473,-0.252667,152.850837,-0.337892,26253,50.843008
22121,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K5(64,...",475545880.0,388.0,40.540935,-0.359168,148.225889,-0.335255,29774,50.842326


In [1]:
import wandb
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/ax1qckwo').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS14_PARAMS_RANDOM_FIXED

In [2]:
run.head(50)

,flops,vkdnw_lambda_1,vkdnw_dim,vkdnw_lambda_6,vkdnw_lambda_8,arch,trainability,vkdnw_entropy,vkdnw_lambda_3,vkdnw_lambda_2,...,vkdnw_lambda_5,_timestamp,vkdnw_lambda_4,vkdnw_lambda_7,vkdnw_chisquare,num_layers,_step,expressivity,progressivity,vkdnw_lambda_0
0,450508056.0,2.999272e-08,244,0.025921,0.552194,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",-0.349545,0.027253,0.000035,1.327641e-06,...,0.006757,1.730193e+09,0.000305,0.107971,152.429907,14,0,119.783098,-0.010152,0.000000e+00
1,457582872.0,0.000000e+00,244,0.003351,0.193395,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",-0.350593,0.008466,0.000002,3.899601e-07,...,0.000456,1.730193e+09,0.000022,0.016570,140.450450,14,1,119.977228,-0.013572,0.000000e+00
2,457582872.0,2.783879e-07,244,0.019030,0.606208,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",-0.352773,0.010086,0.000052,2.409316e-06,...,0.003320,1.730193e+09,0.000744,0.109983,156.056604,14,2,120.039291,-0.012472,0.000000e+00
3,477401804.0,1.169755e-08,306,0.252917,9.226439,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.585437,0.020041,0.000122,3.961710e-06,...,0.049668,1.730193e+09,0.003580,1.967010,132.494118,15,3,-Infinity,-Infinity,0.000000e+00
4,381031544.0,9.205151e-08,244,0.002528,0.087383,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.366132,0.034717,0.000009,6.640306e-07,...,0.000551,1.730193e+09,0.000124,0.017681,161.179487,14,4,117.774245,-0.507675,0.000000e+00
5,434595992.0,1.524530e-07,248,0.003201,0.080045,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.357056,0.007461,0.000009,1.924249e-06,...,0.000723,1.730193e+09,0.000069,0.010745,169.646018,14,5,118.819937,-0.009373,0.000000e+00
6,402400248.0,0.000000e+00,270,0.015578,0.893442,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.351708,0.005039,0.000005,6.091686e-07,...,0.001244,1.730193e+09,0.000054,0.162234,130.349515,15,6,124.559421,-0.722001,0.000000e+00
7,410379800.0,1.011276e-06,274,0.026810,1.205330,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-0.354069,0.004473,0.000083,2.054571e-05,...,0.006652,1.730193e+09,0.001018,0.129387,167.821782,15,7,123.300824,-0.361047,0.000000e+00
8,417704712.0,2.186542e-07,280,0.027110,0.600316,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.449067,0.027124,0.000035,1.698244e-06,...,0.004529,1.730193e+09,0.000893,0.140381,174.209524,14,8,123.344525,-0.250491,0.000000e+00
9,436358424.0,2.217269e-07,244,0.050268,0.602638,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.360748,0.006864,0.000031,1.029203e-06,...,0.008072,1.730193e+09,0.001152,0.130753,173.264151,14,9,119.394996,-0.00894,0.000000e+00


In [6]:
run['arch']

0        SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
1        SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...
2        SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...
3        SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...
4        SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...
                               ...                        
99995    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
99996    SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...
99997    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
99998    SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...
99999    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
Name: arch, Length: 100000, dtype: object

In [7]:
import re
def remove_parentheses(text):
    # Regular expression pattern to match any substring within parentheses
    return re.sub(r"\(.*?\)", "/", text)

In [8]:
list_all = []
for arch in run.arch:
    list_add = remove_parentheses(arch).split('/')
    list_all = list_all + list_add
list_all = set(list_all)

In [9]:
set(list_all)

{'',
 'SuperConvK1BNRELU',
 'SuperConvK3BNRELU',
 'SuperResIDWE1K3',
 'SuperResIDWE1K5',
 'SuperResIDWE1K7',
 'SuperResIDWE2K3',
 'SuperResIDWE2K5',
 'SuperResIDWE2K7',
 'SuperResIDWE4K3',
 'SuperResIDWE4K5',
 'SuperResIDWE4K7',
 'SuperResIDWE6K3',
 'SuperResIDWE6K5',
 'SuperResIDWE6K7'}

In [10]:
(run['arch']).sort_values()

10085    SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K3(10...
7799     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K3(10...
2424     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K7(10...
3013     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE4K3(10...
1420     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE4K3(10...
                               ...                        
2179     SuperConvK3BNRELU(3,96,2,1)SuperResIDWE4K5(96,...
21010    SuperConvK3BNRELU(3,96,2,1)SuperResIDWE4K7(96,...
1877     SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
970      SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
47024    SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
Name: arch, Length: 100000, dtype: object